# 2023/5/26 
# rdkitを用いたボンド情報読み込みクラスread_molのデバックを行う

## まず，obabelでgroからmolへと変換する必要がある．
## ここをやらなくて良いように，CPmake.py smileにmolを作る機能も追加しておいた．
```
obabel -i gro input_GMX.gro -o mol > input_GMX.mol
```

In [1]:
from importlib import reload # >= python 3.4
import ml.atomtype
reload(ml.atomtype)

<module 'ml.atomtype' from '/Users/amano/works/research/dieltools/src/ml/atomtype.py'>

In [2]:
filename_mol="../smiles/pg.acpype/input_GMX.mol"
filename_itp="../smiles/pg.acpype/input_GMX.itp"

In [4]:
test_mol=ml.atomtype.read_mol(filename_mol)
print(" ====== ")
test_itp=ml.atomtype.read_itp(filename_itp)

 -----  ml.read_mol :: parse results... -------
 bonds_list ::  [[1, 7], [1, 0], [2, 3], [2, 8], [2, 1], [3, 9], [4, 2], [5, 0], [6, 1], [10, 4], [11, 4], [12, 4]]
 counter    ::  13
 atom_list  ::  ['O', 'C', 'C', 'O', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
 -----------------------------------------------
 CH bonds...       [[1, 7], [2, 8], [6, 1], [10, 4], [11, 4], [12, 4]]
 CO bonds...       [[1, 0], [2, 3]]
 OH bonds...       [[3, 9], [5, 0]]
 OO bonds...       []
 CC bonds...       [[2, 1], [4, 2]]
 CC ring bonds...  []
 

 ring_bond_index  []
 ch_bond_index    [0, 3, 8, 9, 10, 11]
 oh_bond_index    [5, 7]
 co_bond_index    [1, 2]
 cc_bond_index    [4, 6]
 O atoms (lonepair)...       [0, 3]
 N atoms (lonepair)...       []
 -----  ml.read_itp  :: parse results... -------
 bonds_list ::  [[0, 1], [0, 5], [1, 2], [1, 6], [1, 7], [2, 3], [2, 4], [2, 8], [3, 9], [4, 10], [4, 11], [4, 12]]
 counter    ::  13
 atomic_type::  ['oh', 'c3', 'c3', 'oh', 'c3', 'ho', 'h1', 'h1', 'h1', 'h

In [7]:
#
# * molオブジェクト内にatomオブジェクトが格納されている

# rdkitのmolオブジェクトから原子を取得
print(test_mol.mol_rdkit.GetAtoms())

# 原子が芳香族かどうか
print(test_mol.mol_rdkit.GetAtoms()[0].GetIsAromatic())

print(test_mol.mol_rdkit.GetAtoms()[0].GetIdx()) #get index
print(test_mol.mol_rdkit.GetAtoms()[0].GetAtomicNum()) #get atomic number



False
0


インデックス	atom.GetIdx()
原子番号	atom.GetAtomicNum()
元素記号	atom.GetSymbol()
質量	atom.GetMass()
混成	atom.GetHybridization()
環内原子か否か	atom.IsInRing()
芳香族か否か	atom.GetIsAromatic()
結合リスト	atom.GetBonds()

In [23]:
#
# * 原子の座標を取得する
# https://stackoverflow.com/questions/71915443/rdkit-coordinates-for-atoms-in-a-molecule
import numpy as np

# test_mol.mol_rdkit.GetConformer().GetAtomPosition(0).x

positions_skelton = []
index_tmp = []
for i, atom in enumerate(test_mol.mol_rdkit.GetAtoms()):
    positions = test_mol.mol_rdkit.GetConformer().GetAtomPosition(i)
    # print(atom.GetSymbol(), positions.x, positions.y, positions.z)
    if atom.GetSymbol() != "H":
        print(atom.GetSymbol(), positions.x, positions.y, positions.z)
        positions_skelton.append(np.array([positions.x, positions.y, positions.z]))
        index_tmp.append(i)

positions_skelton=np.array(positions_skelton)
positions_mean = np.mean(positions_skelton, axis=0)
print(positions_mean)

# positions_meanに一番近い原子を探す
distance = np.linalg.norm(positions_skelton - positions_mean,axis=1)
print(distance)

# distanceの中で最小のindexを返す
print(np.argmin(distance))
print(index_tmp)

# 最小のindexを与える原子のindexを返す
print(index_tmp[np.argmin(distance)])

O 0.95 -0.02 0.07
C 0.45 -0.07 -1.27
C 0.95 1.14 -2.05
O 2.38 1.14 -2.05
C 0.47 1.12 -3.49
[ 1.04   0.662 -1.758]
[1.95315335 1.05927711 0.56731649 1.45235946 1.8800234 ]
2
[0, 1, 2, 3, 4]
2


In [52]:
chemmol=test_mol.mol_rdkit
print(chemmol.GetNumAtoms())
for i in chemmol.GetAtoms():
    print(i.GetIdx(), i.GetSymbol(), i.GetAtomicNum())

13
0 O 8
1 C 6
2 C 6
3 O 8
4 C 6
5 H 1
6 H 1
7 H 1
8 H 1
9 H 1
10 H 1
11 H 1
12 H 1
